In [1]:
import ipywidgets as widgets
from ipywidgets import interact
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from dfply import *
import seaborn as sns
import scipy.stats
from datetime import datetime
from scipy.integrate import quad
from dateutil.relativedelta import relativedelta

In [2]:
# 환경설정
plt.style.use('seaborn')
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

In [3]:
# 사용자 함수
# 기준년월생성
def generate_yyyymm(start_yyyymm, end_yyyymm, typ=1):
    '''
        Description
        -----------
        기준년월 리스트를 %Y%m 형식으로 생성
        
        Example
        -------
        yyyymm_list = generate_yyyymm((2017, 1), (2019, 12))
        
    '''
    (start_year, start_month), (end_year, end_month) = start_yyyymm, end_yyyymm
    yyyymm_list = []
    year_month = datetime(start_year, start_month, 1)
    while(year_month <= datetime(end_year, end_month, 1)):
        if typ==1:
            yyyymm_list.append(year_month.strftime('%Y%m'))
        elif typ==0:
            yyyymm_list.append(year_month)
        year_month += relativedelta(months=1)
    return yyyymm_list

# 요약
def summary(data, typ=0):
    if typ==0:
        count = len(data)
        mean = data.mean()
        std = data.std()
        min_ = data.min()
        max_ = data.max()
        q = list(np.quantile(data, q=[0.005, 0.025, 0.05, 0.25, 0.50, 0.75, 0.95, 0.975, 0.995]))
    elif typ==1:
        count = np.nan
        mean = data.mean()
        std = data.std()
        min_ = np.nan
        max_ = np.nan
        q = [data.ppf(0.005), data.ppf(0.025), data.ppf(0.05), data.ppf(0.25), data.ppf(0.50), data.ppf(0.75), data.ppf(0.95), data.ppf(0.975), data.ppf(0.995)]
    return pd.Series(data=[count, mean, std, min_] + q + [max_], index=['count', 'mean', 'std', 'min', '0.5%', '2.5%', '5%', '25%', '50%', '75%', '95%', '97.5%', '99.5%', 'max'])

# Fitting
def fit(data):
    dist_names = [
        "alpha","anglit","arcsine","beta","betaprime","bradford","burr","cauchy",
        "chi","chi2","cosine","dgamma","dweibull","erlang","expon","exponweib","exponpow",
        "f","fatiguelife","fisk","foldcauchy","foldnorm","frechet_r","frechet_l",
        "genlogistic","genpareto","genexpon","genextreme","gausshyper","gamma","gengamma",
        "genhalflogistic","gilbrat","gompertz","gumbel_r","gumbel_l","halfcauchy","halflogistic",
        "halfnorm","hypsecant","invgamma","invgauss","invweibull","johnsonsb","johnsonsu",
        "ksone","kstwobign","laplace","logistic","loggamma","loglaplace","lognorm","lomax",
        "maxwell","mielke","nakagami","ncx2","ncf","nct","norm","pareto","pearson3","powerlaw",
        "powerlognorm","powernorm","rdist","reciprocal","rayleigh","rice","recipinvgauss",
        "semicircular","t","triang","truncexpon","truncnorm","tukeylambda","uniform","vonmises",
        "wald","weibull_min","weibull_max","wrapcauchy",
    ]
    
    result = []
    for name in dist_names:
        dist = getattr(scipy.stats, name)
        params = dist.fit(data, floc=0)
        model = dist(*params[:-2], loc=params[-2], scale=params[-1])
        loglik = np.log(model.pdf(data)).sum()
        result.append([name, loglik])

    result = pd.DataFrame(result, columns=['분포명', '로그우도']) \
        .sort_values(by='로그우도', ascending=False) \
        .reset_index(drop=True)
    return result

In [4]:
# 데이터 불러오기
실적 = pd.read_excel('data/동산종합보험_실적_20200702.xlsx', dtype={'증권번호': str, 'FY년월': str, 'UY년월': str})
사고 = pd.read_excel('data/동산종합보험_사고_20200702.xlsx', dtype={'증권번호': str, '사고접수번호': str, '사고발생일자': str, '사고접수일자': str})
증권정보 = pd.read_excel('data/동산종합보험_증권정보_20200702.xlsx', dtype={'증권번호': str, '계약자고객번호': str})
# 상품코드 = pd.read_excel('data/상품코드.xlsx')
# 섹션코드 = pd.read_excel('data/섹션코드.xlsx')
# 판매담보코드 = pd.read_excel('data/판매담보코드.xlsx')
# 보험금처리구분코드 = pd.read_excel('data/보험금처리구분코드.xlsx')
# 사고목적구분코드 = pd.read_excel('data/사고목적구분코드.xlsx')
# POOL구분코드 = pd.read_excel('data/POOL구분코드.xlsx')

In [5]:
# UY 및 FY 범위
uy_min = 실적.query('UY != " "')['UY년월'].min()
uy_max = 실적.query('UY != " "')['UY년월'].max()
fy_min = 실적['FY년월'].min()
fy_max = 실적['FY년월'].max()
print(f'UY: {uy_min} ~ {uy_max}')
print(f'FY: {fy_min} ~ {fy_max}')

UY: 199906 ~ 202005
FY: 200901 ~ 202005


In [6]:
# 변수 설정
FY시작년월, FY종료년월 = datetime(2012, 1, 1), datetime(2020, 5, 1)
UY시작년월, UY종료년월 = datetime(2012, 1, 1), datetime(2020, 5, 1)
UY년월 = pd.Series(generate_yyyymm((UY시작년월.year, UY시작년월.month), (UY종료년월.year, UY종료년월.month), 1))
FY년월 = pd.Series(generate_yyyymm((FY시작년월.year, FY시작년월.month), (FY종료년월.year, FY종료년월.month), 1))

In [7]:
# 데이터 가공
# 실적 데이터
실적_가공 = 실적 \
    >> mask(X.POOL구분코드 == ' ') \
    >> drop(X.POOL구분코드) \
    >> mask(X.UY년월 >= UY시작년월.strftime('%Y%m'))
증권번호_유효 = 실적_가공.groupby('증권번호')[['원수보험료', '가입금액']].sum().query('원수보험료 > 0 and 가입금액 >= 0').index
증권번호_무효 = 실적_가공.query('증권번호 not in @증권번호_유효')['증권번호'].unique()
실적_가공 = 실적_가공 \
    >> mask(X.증권번호.isin(증권번호_유효))
증권번호_UY = 실적_가공[['증권번호', 'UY년월']].drop_duplicates()
가입금액_증권별 = 실적_가공.groupby('증권번호')[['가입금액']].sum().reset_index()
가입금액_증권별['가입금액'] = 가입금액_증권별['가입금액'].astype(float)

# 증권정보 데이터
증권정보_가공 = 증권정보 \
    >> mask(X.증권번호.isin(증권번호_유효)) \
    >> drop(X.입출금고객번호, X.법인사업자등록번호, X.법인등록번호) \
    >> mask(X.보험기간시작일자 != ' ', X.보험기간종료일자 != ' ')

증권정보_가공 = 증권정보_가공.merge(증권번호_UY, on='증권번호', how='left')
증권정보_가공['보험기간시작일자'] = pd.to_datetime(증권정보_가공['보험기간시작일자'], format='%Y%m%d')
증권정보_가공['보험기간종료일자'] = pd.to_datetime(증권정보_가공['보험기간종료일자'], format='%Y%m%d')
# 증권정보_가공 = 증권정보_가공[['증권번호', 'UY년월', '보험기간시작일자', '보험기간종료일자']].drop_duplicates()
증권정보_보험기간종료일자 = 증권정보_가공.groupby('증권번호')['보험기간종료일자'].max().reset_index()  # 보험기간종료일자 최대값 처리
증권정보_가공 = 증권정보_가공[['증권번호', 'UY년월', '보험기간시작일자']].drop_duplicates() \
    .merge(증권정보_보험기간종료일자, on='증권번호', how='left')
증권정보_가공['보험기간'] = (증권정보_가공['보험기간종료일자'] - 증권정보_가공['보험기간시작일자']).apply(lambda x: x.days)

# 실적에 증권정보 left join (Validation 하고 할 것!)
실적_가공 = 실적_가공.merge(증권정보_가공, on=['증권번호', 'UY년월'], how='left')

# 사고 데이터
사고키 = ['사고접수번호', '증권번호']
기준년월 = 사고['마감년월'].max()
사고_가공 = 사고 \
    >> mask(X.증권번호.isin(증권번호_유효))

손해액_사고별 = 사고_가공.groupby(사고키)[['당사지급보험금', '당사손해조사비']].sum().reset_index()
개별추산액_사고별 = 사고_가공.query('마감년월 == @FY종료년월.strftime("%Y%m")').groupby(['사고접수번호', '증권번호'])['당사후기추산보험금'].sum().reset_index()
사고발생일자_사고별 = 사고_가공[사고키 + ['사고발생일자']].drop_duplicates()
사고접수일자_사고별 = 사고_가공.groupby(사고키)['사고접수일자'].min().reset_index() # 최소값 맵핑
손해액_사고별 = 손해액_사고별.merge(개별추산액_사고별, on=사고키, how='outer')
손해액_사고별[['당사지급보험금', '당사손해조사비', '당사후기추산보험금']] = 손해액_사고별[['당사지급보험금', '당사손해조사비', '당사후기추산보험금']].fillna(0).astype(float)
손해액_사고별.eval('원수발생손해액 = 당사지급보험금 + 당사손해조사비 + 당사후기추산보험금', inplace=True)
손해액_사고별.columns = 사고키 + ['원수보험금', '원수손해조사비', '원수개별추산액', '원수손해액']
손해액_사고별 = 손해액_사고별.merge(증권번호_UY, on='증권번호', how='left')
손해액_사고별 = 손해액_사고별.merge(사고발생일자_사고별, on=사고키, how='left')
손해액_사고별 = 손해액_사고별.merge(사고접수일자_사고별, on=사고키, how='left')
손해액_사고별['사고발생일자'] = pd.to_datetime(손해액_사고별['사고발생일자'], format='%Y%m%d')
손해액_사고별['사고접수일자'] = pd.to_datetime(손해액_사고별['사고접수일자'], format='%Y%m%d')
손해액_사고별 = 손해액_사고별.merge(가입금액_증권별, on='증권번호', how='left')
손해액_사고별['AY년월'] = 손해액_사고별['사고발생일자'].apply(lambda x: f'{x.year}{x.month:02}')
손해액_사고별['보고지연'] = (손해액_사고별['사고접수일자'] - 손해액_사고별['사고발생일자']).apply(lambda x: x.days)
손해액_사고별 = 손해액_사고별[사고키 + ['AY년월', 'UY년월', '사고발생일자', '사고접수일자', '보고지연', '가입금액', '원수보험금', '원수개별추산액', '원수손해조사비', '원수손해액']]
손해액_사고별 = 손해액_사고별.query('원수손해액 > 0') \
    .sort_values(by='원수손해액', ascending=False) \
    .reset_index(drop=True)
손해액_사고별['건수비중_누적'] = (손해액_사고별.index+1)/len(손해액_사고별)
손해액_사고별['손해액비중_누적'] = 손해액_사고별['원수손해액'].cumsum()/손해액_사고별['원수손해액'].sum()

# 사고내용
사고내용 = 사고.groupby(['사고접수번호', '증권번호'])['마감년월'].max().reset_index() \
    .merge(사고[['사고접수번호', '증권번호', '마감년월', '사고내용']].drop_duplicates(), on=['사고접수번호', '증권번호', '마감년월'], how='left')

In [15]:
# 무효처리 건
실적_무효처리 = 실적.query('증권번호 in @증권번호_무효').groupby('FY')[['원수보험료', '원수경과보험료', '원수발생손해액']].sum()
실적_무효처리

,원수보험료,원수경과보험료,원수발생손해액
FY,,,
2012,0,0,0
2013,98300,30522,0
2014,58100,32703,0
2015,200901700,16412121,0
2016,-201058100,-16475346,3382000
2017,3365880,2268511,0
2018,-3365880,-2268511,0
2019,17744240,0,106000
2020,-17744240,0,0


In [16]:
# Validation
# 증권번호 → UY년월 injectivity 체크
증권번호_UY_체크 = 증권번호_UY.groupby('증권번호')['UY년월'].size().max() == 1
print(f'UY년월 Injectivity: {증권번호_UY_체크}')

# 실적 및 사고 데이터 지급보험금 맞는지 체크
원수보험금_체크 = 실적_가공['원수보험금'].sum() - 손해액_사고별['원수보험금'].sum()
print(f'원수보험금 합계: {원수보험금_체크}')

# 실적 및 사고 데이터 손해조사비 맞는지 체크 (201001 정도부터 해야 맞음)
# 조금 차이남
손해조사비_체크 = 실적_가공['원수손해조사비'].sum() - 손해액_사고별['원수손해조사비'].sum()
print(f'손해조사비 합계: {손해조사비_체크}')

# 실적 및 사고 데이터 손해액 맞는지 체크
손해액_체크 = 실적_가공['원수발생손해액'].sum() - 손해액_사고별['원수손해액'].sum()
print(f'손해액 합계: {손해액_체크}')

# 사고키 → 사고발생일자 injectivity 체크
사고발생일자_체크 = 사고발생일자_사고별.groupby(사고키).size().max() == 1
print(f'사고발생일자 Injectivity: {사고발생일자_체크}')

# 사고키 → 사고접수일자 injectivity 체크
사고접수일자_체크 = 사고접수일자_사고별.groupby(사고키).size().max() == 1
print(f'사고접수일자 Injectivity: {사고접수일자_체크}')

# 증권번호 → 계약자고객번호 injectivity 체크
# 증권번호 → 보험기간시작일자 injectivity 체크
# 증권번호 → 보험기간시작일자 injectivity 체크
증권별정보_체크 = 증권정보_가공.groupby('증권번호').size().max() == 1
print(f'계약자고객번호, 보험기간시작일자, 보험기간종료일자 Injectivity: {증권별정보_체크}')

# 증권정보 증권번호 surjectivity 체크
증권누락여부_체크 = sum(~실적_가공['증권번호'].isin(증권정보_가공['증권번호'])) == 0
print(f'증권번호 Surjectivity: {증권누락여부_체크}')

UY년월 Injectivity: True
원수보험금 합계: 0.0
손해조사비 합계: -3999.9900000095367
손해액 합계: -3999.9899978637695
사고발생일자 Injectivity: True
사고접수일자 Injectivity: True
계약자고객번호, 보험기간시작일자, 보험기간종료일자 Injectivity: True
증권번호 Surjectivity: True


In [17]:
# 예외발생
사고발생일자_예외발생 = 사고발생일자_사고별.groupby(사고키).size().reset_index(name='건수').query('건수 > 1')
증권정보_예외발생 = 증권정보_가공.groupby('증권번호').size().reset_index(name='건수').query('건수 > 1')
증권정보_예외발생

,증권번호,건수


In [18]:
# 예외처리 내역
# 증권번호

In [19]:
# UY별 실적 가공
# 계약건수 및 사고건수
계약건수_UY별 = 실적_가공.groupby('UY년월')['증권번호'].apply(lambda x: len(x.unique())).reset_index(name='계약건수')
사고건수_UY별 = 손해액_사고별.groupby('UY년월').size().reset_index(name='사고건수')

# 집계
실적_UY별 = 실적_가공.groupby('UY년월')[['가입금액', '원수보험료', '원수경과보험료', '원수보험금', '원수OS증감액', '원수손해조사비', '원수발생손해액']].sum() \
    .eval('원수미경과보험료 = 원수보험료-원수경과보험료') \
    .reset_index() \
    .merge(계약건수_UY별, on='UY년월', how='outer') \
    .merge(사고건수_UY별, on='UY년월', how='outer') \
    .merge(UY년월.to_frame('UY년월'), how='outer') \
    .fillna(0) \
    .sort_values(by='UY년월')
실적_UY별.columns = ['UY년월', '가입금액', '수입보험료', '경과보험료', '보험금', '개별추산액', '손해조사비', '손해액', '미경과보험료', '계약건수', '사고건수']
실적_UY별 = 실적_UY별[['UY년월', '계약건수', '사고건수', '가입금액', '수입보험료', '경과보험료', '미경과보험료', '보험금', '개별추산액', '손해조사비', '손해액']]

# 2차 가공
실적_UY별['경과월'] = 실적_UY별['UY년월'].apply(lambda x: datetime.strptime(x, '%Y%m')) \
    .apply(lambda x: 12*relativedelta(datetime(UY종료년월.year, UY종료년월.month, 1), x).years+relativedelta(datetime(UY종료년월.year, UY종료년월.month, 1), x).months)
실적_UY별['계약보정계수'] = 실적_UY별['경과월'].apply(lambda x: np.fmin((x+0.5)/12, 1))
실적_UY별 = 실적_UY별.eval('보정계약건수 = 계약건수*계약보정계수') \
    .eval('보험료_가입금액당 = 수입보험료/가입금액') \
    .eval('보험료_계약건당 = 수입보험료/계약건수') \
    .eval('손해액_사고건당 = 손해액/사고건수') \
    .eval('사고율_계약건당 = 사고건수/계약건수') \
    .eval('사고율_보험료당 = 사고건수/경과보험료') \
    .eval('손해율 = 손해액/경과보험료') \
    .fillna(0)
실적_UY별.insert(0, 'UY', 실적_UY별['UY년월'].str.slice(0,4))

실적_UY별

,UY,UY년월,계약건수,사고건수,가입금액,수입보험료,경과보험료,미경과보험료,보험금,개별추산액,...,손해액,경과월,계약보정계수,보정계약건수,보험료_가입금액당,보험료_계약건당,손해액_사고건당,사고율_계약건당,사고율_보험료당,손해율
0,2012,201201,134,26.0,281131568008,271595300,271595300,0,20215346,0,...,25274826,100,1.000000,134.000000,0.000966,2.026831e+06,9.721087e+05,0.194030,9.573067e-08,0.093061
1,2012,201202,177,28.0,329653569576,198013324,198013324,0,283243425,0,...,292354260,99,1.000000,177.000000,0.000601,1.118719e+06,1.044122e+07,0.158192,1.414046e-07,1.476437
2,2012,201203,163,1244.0,1041562521331,394712617,394712617,0,378634355,0,...,411816607,98,1.000000,163.000000,0.000379,2.421550e+06,3.310423e+05,7.631902,3.151660e-06,1.043333
3,2012,201204,221,51.0,2097056263269,242488949,242488949,0,187195452,0,...,198673452,97,1.000000,221.000000,0.000116,1.097235e+06,3.895558e+06,0.230769,2.103189e-07,0.819309
4,2012,201205,154,41.0,1310728775452,336144522,336144522,0,140860486,0,...,154702886,96,1.000000,154.000000,0.000256,2.182757e+06,3.773241e+06,0.266234,1.219713e-07,0.460227
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,2020,202001,67,0.0,451130483206,171845190,73136429,98708761,0,0,...,0,4,0.375000,25.125000,0.000381,2.564854e+06,0.000000e+00,0.000000,0.000000e+00,0.000000
97,2020,202002,89,27.0,108785169616,145408956,60444180,84964776,12029044,14300000,...,31110044,3,0.291667,25.958333,0.001337,1.633808e+06,1.152224e+06,0.303371,4.466931e-07,0.514690
98,2020,202003,53,9.0,111683334425,137573060,42469519,95103541,2042740,8100000,...,11316340,2,0.208333,11.041667,0.001232,2.595718e+06,1.257371e+06,0.169811,2.119167e-07,0.266458
99,2020,202004,66,7.0,102305485762,216335754,38539980,177795774,1363500,16000000,...,18353500,1,0.125000,8.250000,0.002115,3.277814e+06,2.621929e+06,0.106061,1.816296e-07,0.476220


In [20]:
# 증권별 실적 가공
# 계약건수 및 사고건수
사고건수_증권별 = 손해액_사고별.groupby('증권번호').size().reset_index(name='사고건수')

# 집계
실적_증권별 = 실적_가공.groupby(['증권번호', '보험기간시작일자', '보험기간종료일자', '보험기간'])[['가입금액', '원수보험료', '원수경과보험료', '원수보험금', '원수OS증감액', '원수손해조사비', '원수발생손해액']].sum().reset_index()
실적_증권별.columns = ['증권번호', '보험기간시작일자', '보험기간종료일자', '보험기간', '가입금액', '수입보험료', '경과보험료', '보험금', '개별추산액', '손해조사비', '손해액']

# 2차 가공
실적_증권별['경과기간'] = np.fmin(실적_증권별['보험기간'], 실적_증권별['보험기간시작일자'].apply(lambda x: (UY종료년월 + relativedelta(months=1, days=-1) - x).days))
실적_증권별 = 실적_증권별 \
    .eval('미경과기간 = 보험기간-경과기간') \
    .merge(사고건수_증권별, on='증권번호', how='left') \
    .eval('미경과보험료 = 수입보험료-경과보험료') \
    .eval('요율_가입금액당 = 수입보험료/가입금액') \
    .eval('손해율 = 손해액/경과보험료') \
    .eval('손해액_사고당 = 손해액/사고건수') \
    .eval('손해액_가입금액대비 = 손해액/가입금액') \
    .fillna(0)

실적_증권별

,증권번호,보험기간시작일자,보험기간종료일자,보험기간,가입금액,수입보험료,경과보험료,보험금,개별추산액,손해조사비,손해액,경과기간,미경과기간,사고건수,미경과보험료,요율_가입금액당,손해율,손해액_사고당,손해액_가입금액대비
0,120140027142,2014-10-06,2017-10-06,1096,248489000,284400,284400,0,0,0,0,1096,0,0.0,0,0.001145,0.0,0.0,0.0
1,120140029363,2014-10-07,2017-10-07,1096,307251000,353500,353500,0,0,0,0,1096,0,0.0,0,0.001151,0.0,0.0,0.0
2,120140029375,2014-10-07,2017-10-07,1096,102389000,117200,117200,0,0,0,0,1096,0,0.0,0,0.001145,0.0,0.0,0.0
3,120140029623,2014-10-07,2017-10-07,1096,140221000,161600,161600,0,0,0,0,1096,0,0.0,0,0.001152,0.0,0.0,0.0
4,120140029823,2014-10-07,2014-10-14,7,137400000,349100,349100,0,0,0,0,7,0,0.0,0,0.002541,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11509,501140001553000,2014-10-02,2015-10-02,365,550000000,1789100,1789100,0,0,0,0,365,0,0.0,0,0.003253,0.0,0.0,0.0
11510,501140001554000,2014-10-02,2015-10-02,365,34003511,47600,47600,0,0,0,0,365,0,0.0,0,0.001400,0.0,0.0,0.0
11511,501140001555000,2014-10-02,2017-10-02,1096,147481000,169600,169600,0,0,0,0,1096,0,0.0,0,0.001150,0.0,0.0,0.0
11512,501140001556000,2014-10-02,2016-10-02,731,56522000,68300,68300,0,0,0,0,731,0,0.0,0,0.001208,0.0,0.0,0.0


In [21]:
# 보고지연 피팅 수행
# 보고지연_데이터 = np.array(손해액_사고별['보고지연'])
# 보고지연_모델목록 = fit(보고지연_데이터)

In [22]:
# 분포 선택 (보고지연)
# dist = getattr(scipy.stats, 'bradford')
# 보고지연_모수 = dist.fit(보고지연_데이터, floc=0)
# 보고지연_모델 = dist(*보고지연_모수[:-2], loc=보고지연_모수[-2], scale=보고지연_모수[-1])
# x = np.linspace(0, 보고지연_데이터.max(), 1000)
# px = 보고지연_모델.pdf(x)

# fig, ax = plt.subplots(2,1, figsize=(16,10))
# ax[0].hist(보고지연_데이터, bins=50, density=True, alpha=0.75, label='데이터')
# ax[0].plot(x, px, color='black')
# ax[0].fill_between(x, px, alpha=0.5, color='yellow', label='피팅')
# ax[0].legend()
# scipy.stats.probplot(보고지연_데이터, dist=dist, sparams=보고지연_모수, plot=ax[1])
# plt.tight_layout()
# plt.show()
# print(f'로그우도: {sum(np.log(보고지연_모델.pdf(보고지연_데이터)))}')

In [23]:
# 보고지연 모델링 결과 요약
# 보고지연_데이터_요약 = summary(보고지연_데이터, typ=0).to_frame('보고지연_데이터')
# 보고지연_모델_요약 = summary(보고지연_모델, typ=1).to_frame('보고지연_모델')

In [24]:
# FY(AY)별 실적 가공
# 보고지연 보정 안 함
# 사고건수, AY손해액
사고건수_AY별 = 손해액_사고별.groupby('AY년월').size().reset_index(name='사고건수')
손해액_AY별 = 손해액_사고별.groupby('AY년월')['원수손해액'].sum().reset_index(name='AY손해액')
손해액_AY별 = 손해액_AY별.merge(사고건수_AY별, on='AY년월', how='outer') \
    .rename(columns={'AY년월': 'FY년월'})

# 집계
실적_FY별 = 실적_가공.groupby(['FY년월'])[['원수보험료', '원수경과보험료', '원수보험금', '원수발생손해액']].sum().astype(float) \
    .reset_index() \
    .merge(FY년월.to_frame('FY년월'), how='outer') \
    .merge(손해액_AY별, on='FY년월', how='outer') \
    .fillna(0) \
    .sort_values(by='FY년월')
실적_FY별.columns = ['FY년월', '수입보험료', '경과보험료', '보험금', 'FY손해액', 'AY손해액', '사고건수']

# 2차 가공
실적_FY별['경과월'] = 실적_FY별['FY년월'].apply(lambda x: datetime.strptime(x, '%Y%m')) \
    .apply(lambda x: 12*relativedelta(datetime(FY종료년월.year, FY종료년월.month, 1), x).years+relativedelta(datetime(FY종료년월.year, FY종료년월.month, 1), x).months)
# 실적_FY별['사고보정계수'] = 실적_FY별['경과월'].apply(lambda i: 30/quad(보고지연_모델.cdf, 30*i, 30*(i+1))[0])
실적_FY별['사고보정계수'] = 1
실적_FY별 = 실적_FY별.eval('보정사고건수 = 사고건수*사고보정계수') \
    .eval('FY손해율 = FY손해액/경과보험료') \
    .eval('보정사고율_보험료당 = 보정사고건수/경과보험료') \
    .eval('AY손해액_사고당 = AY손해액/사고건수') \
    .eval('AY손해율 = AY손해액/경과보험료') \
    .fillna(0) \
    .reset_index(drop=True)
실적_FY별.insert(0, 'FY', 실적_FY별['FY년월'].str.slice(0,4))

실적_FY별

,FY,FY년월,수입보험료,경과보험료,보험금,FY손해액,AY손해액,사고건수,경과월,사고보정계수,보정사고건수,FY손해율,보정사고율_보험료당,AY손해액_사고당,AY손해율
0,2012,201201,326989376.0,10749870.0,0.0,5000000.0,8623044.0,9,100,1,9,0.465122,8.372194e-07,9.581160e+05,0.802153
1,2012,201202,210004745.0,37450836.0,3049000.0,11971000.0,12162482.0,10,99,1,10,0.319646,2.670167e-07,1.216248e+06,0.324759
2,2012,201203,243011643.0,59767075.0,1465316.0,14796.0,24721420.0,16,98,1,16,0.000248,2.677059e-07,1.545089e+06,0.413629
3,2012,201204,386957644.0,99618263.0,6016506.0,56618506.0,36729042.0,136,97,1,136,0.568355,1.365212e-06,2.700665e+05,0.368698
4,2012,201205,353430258.0,121862320.0,12686426.0,6285926.0,58072541.0,122,96,1,122,0.051582,1.001130e-06,4.760044e+05,0.476542
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,2020,202001,174208307.0,297673687.0,68234003.0,212095587.0,167639777.0,37,4,1,37,0.712510,1.242972e-07,4.530805e+06,0.563166
97,2020,202002,160589642.0,272913798.0,116571442.0,212311538.0,157207592.0,53,3,1,53,0.777944,1.942005e-07,2.966181e+06,0.576034
98,2020,202003,124039275.0,285296579.0,337942200.0,479418001.0,152493443.0,43,2,1,43,1.680420,1.507203e-07,3.546359e+06,0.534508
99,2020,202004,252639571.0,273445402.0,313876577.0,294028088.0,251138480.0,42,1,1,42,1.075272,1.535956e-07,5.979488e+06,0.918423


In [25]:
# 손해액 피팅 수행
# 손해액_데이터 = np.array(손해액_사고별['원수손해액'])[300:]
# 손해액_모델목록 = fit(손해액_데이터)
# 손해액_데이터_고액 = np.array(손해액_사고별['원수손해액'])[:300]
# 손해액_모델목록_고액 = fit(손해액_데이터)
# 손해액_모델목록_고액

In [26]:
# # 분포 선택 (손해액)
# dist = getattr(scipy.stats, 'genpareto')
# 손해액_모수 = dist.fit(손해액_데이터, floc=0)
# 손해액_모델 = dist(*손해액_모수[:-2], loc=손해액_모수[-2], scale=손해액_모수[-1])
# x = np.linspace(0, np.quantile(손해액_데이터, q=0.995), 1000)
# px = 손해액_모델.pdf(x)

# fig, ax = plt.subplots(2,1, figsize=(16,10))
# ax[0].hist(손해액_데이터, bins=50, density=True, alpha=0.75, label='데이터')
# ax[0].plot(x, px, color='black')
# ax[0].fill_between(x, px, alpha=0.5, color='yellow', label='피팅')
# ax[0].legend()
# ax[0].set_xlim([0, np.quantile(손해액_데이터, q=0.995)])
# scipy.stats.probplot(손해액_데이터, dist=dist, sparams=손해액_모수, plot=ax[1])
# plt.tight_layout()
# plt.savefig('img/손해액분포')
# plt.show()
# print(f'로그우도: {sum(np.log(손해액_모델.pdf(손해액_데이터)))}')

In [27]:
# # 손해액 모델링 결과 요약
# 손해액_데이터_요약 = summary(손해액_데이터, typ=0).to_frame('손해액_데이터')
# 손해액_모델_요약 = summary(손해액_모델, typ=1).to_frame('손해액_모델')
# # pd.concat([손해액_모델_요약, 손해액_데이터_요약], axis=1)

In [28]:
# # 빈도 추정 (미경과보험료)
# 미경과보험료 = 실적_UY별['미경과보험료'].sum()
# 경과보험료당_사고건수 = 실적_FY별['보정사고건수'].sum()/실적_FY별['경과보험료'].sum()
# 빈도_미경과분 = 미경과보험료*경과보험료당_사고건수
# dist = getattr(scipy.stats, 'poisson')
# 빈도_모델 = dist(빈도_미경과분)

In [29]:
# # 시뮬레이션
# np.random.seed(20200623)
# 시나리오개수 = 10000
# 총손해액_시뮬레이션 = np.array(list(map(lambda n: 손해액_모델.rvs(n).sum(), 빈도_모델.rvs(시나리오개수))))
# 손해율_시뮬레이션 = 총손해액_시뮬레이션/미경과보험료

In [30]:
# 시뮬레이션 시각화
# fig, ax = plt.subplots(1,1, figsize=(16,5))
# ax.hist(손해율_시뮬레이션, bins=시나리오개수, density=True)
# ax.set_xlim([0, np.quantile(손해율_시뮬레이션, q=0.995)])
# plt.tight_layout()
# plt.savefig('img/총손해액시뮬레이션')
# plt.show()

In [31]:
# # 시뮬레이션 결과 요약
# 총손해액_시뮬레이션_요약 = summary(총손해액_시뮬레이션, typ=0).to_frame('총손해액_시뮬레이션')
# 손해율_시뮬레이션_요약 = summary(손해율_시뮬레이션, typ=0).to_frame('손해율_시뮬레이션')

In [32]:
# # 요약 집계
# # 요약_집계 = pd.concat([보고지연_데이터_요약, 보고지연_모델_요약, 손해액_데이터_요약, 손해액_모델_요약, 총손해액_시뮬레이션_요약, 손해율_시뮬레이션_요약], axis=1).T
# 요약_집계 = pd.concat([손해액_데이터_요약, 손해액_모델_요약, 총손해액_시뮬레이션_요약, 손해율_시뮬레이션_요약], axis=1).T

In [33]:
# 요약_집계

In [34]:
# 출력
now = datetime.now().strftime('%Y%m%d%H%M%S')
with pd.ExcelWriter(f'result/동산종합보험_분석결과_{now}.xlsx', 'xlsxwriter') as writer:
    실적_FY별.to_excel(writer, '실적_FY별', index=False)
    실적_UY별.to_excel(writer, '실적_UY별', index=False)
    사고내용.to_excel(writer, '사고내용', index=False)
    손해액_사고별.to_excel(writer, '손해액_사고별', index=False)
#     증권정보_가공.to_excel(writer, '증권정보', index=False)
    실적_증권별.to_excel(writer, '실적_증권별', index=False)
#     요약_집계.to_excel(writer, '요약', index=True)